In [25]:
import praw

# Initialize PRAW with your credentials
reddit = praw.Reddit(
    client_id='GGoNNF-RLZRl4R5R3p-qMQ',  # Your client_id
    client_secret='kmBes1g4S2iClV9f_6X78Gr1DPCZLw',  # Your client_secret
    user_agent='ProductReviewBot/0.1 by JeanLuucGodard',  # Your user_agent
)

# Define the subreddit and keyword
subreddit_name = 'samsunggalaxy'  # You can change this to any specific subreddit
search_keyword = 'S24'  # Replace this with your keyword or brand name

# Initialize an empty string to store all the content
all_content = ""

# Search for posts with the keyword in the specified subreddit
posts = reddit.subreddit(subreddit_name).search(search_keyword, limit=10)  # You can change the limit

# Process the posts and append the content to the all_content variable
for post in posts:
    all_content += f"Title: {post.title}\n"
    all_content += f"URL: {post.url}\n"
    all_content += f"Score: {post.score}\n"
    all_content += f"Created: {post.created_utc}\n"
    all_content += f"Author: {post.author}\n"
    all_content += f"Text: {post.selftext[:500]}...\n"  # Only the first 500 characters of the post
    all_content += "\n---\n"
    
    # Fetch comments for each post and append them to all_content
    post.comments.replace_more(limit=0)  # This removes 'More comments' objects for better processing
    for comment in post.comments.list():
        all_content += f"Comment by {comment.author}: {comment.body[:200]}...\n"  # First 200 characters of comment
    all_content += "\n===\n"

# Now, all_content contains the entire text for the posts and comments
print(all_content)  # You can print or further process this string as needed


Title: Worth buying S24 now or wait for S25?
URL: https://www.reddit.com/r/samsunggalaxy/comments/1hircma/worth_buying_s24_now_or_wait_for_s25/
Score: 8
Created: 1734722202.0
Author: kdrxyz
Text: I did buy an S24 when it was launched. Returned it because the image/photo quality (not camera quality) was worse than the S21 I was using. Over the year, my S21 has started to develop issues (slow performance, low battery etc.). So, I have to buy a new phone soon. Should I wait for S25 (launching Jan) or buy S24 with a holiday discount?

On a similar note, has Samsung improved S24 photo quality with newer software updates?...

---
Comment by Old_Atmosphere_651: The new Snapdragon 8 elite chip is a pretty substantial upgrade, bringing 30% increased performance and efficiency. 

Even if you're not interested in sheer performance upgrades, the battery saving an...
Comment by qualitative_balls: I took a chance on a renewed S24+ on Amazon for $600.

It literally... Looks brand new. Like, it could 

In [26]:
import tiktoken  # Install this package for token counting

# Function to estimate the number of tokens in a text
def num_tokens_from_messages(messages):
    encoding = tiktoken.get_encoding("o200k_base")  # Use appropriate GPT model encoding
    tokens = 0
    for message in messages:
        tokens += len(encoding.encode(message['content']))
    return tokens

# Function to ensure the input doesn't exceed the token limit
def ensure_tokens_within_limit(all_content, prompt_template, token_limit=128000):
    # Initialize the list of messages
    messages = [
        {"role": "system", "content": prompt_template},
        {"role": "user", "content": all_content}
    ]
    
    # Calculate the number of tokens for the current messages
    tokens_used = num_tokens_from_messages(messages)
    
    # If the tokens exceed the limit, trim the content
    while tokens_used > token_limit:
        print(f"Token limit exceeded! Trimming content...")
        # Trim the content to reduce token count (you can adjust this trimming logic)
        all_content = all_content[:int(len(all_content) * 0.9)]  # Reduce content size by 10%
        messages[1]['content'] = all_content
        
        tokens_used = num_tokens_from_messages(messages)
        print(f"Tokens used after trimming: {tokens_used}")
    
    return all_content, messages

In [27]:
from dotenv import load_dotenv
import os
import openai

# Load environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')

prompt_template = """
Based on the Reddit scraped content provided, please generate a comprehensive and insightful summary for a Brand Manager. The summary should include:

1. **Sentiment Analysis**: Describe the overall sentiment expressed by users in the content—whether it's positive, negative, or neutral towards the brand/product. Mention key emotions expressed, if relevant (e.g., frustration, excitement, satisfaction, etc.).
  
2. **Key Themes**: Identify and summarize the main topics or themes discussed across the posts (e.g., product features, customer service, pricing, competitors, etc.). Point out any emerging trends or shifts in opinion.

3. **Common Issues**: Highlight any recurring complaints or negative feedback about the brand/product. Include the specific aspects of the product or service that users are dissatisfied with.

4. **Suggestions and Recommendations**: Based on the comments, provide any constructive suggestions or recommendations for improvement. This could include ideas on product improvement, marketing strategies, or customer support enhancements.

5. **Competitor Mentions**: Note any discussions or comparisons with competitor brands/products. How does your brand stand in relation to competitors based on user opinions?

6. **Engagement Insights**: If applicable, mention any significant discussions, highly upvoted comments, or trending posts that could offer further insights into consumer behavior or opinions on the brand.

Please provide the summary in a concise yet thorough manner, ensuring it's tailored for a Brand Manager to quickly understand the sentiment, key issues, and actionable insights for brand strategy improvement.

"""

from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI()

# Ensure the content stays within the token limit
all_content, messages = ensure_tokens_within_limit(all_content, prompt_template)


completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": prompt_template},
    {"role": "user", "content": all_content}
  ])

In [28]:
report = completion.choices[0].message.content

In [30]:
from IPython.display import Markdown

# Display Markdown
display(Markdown(report))


### Brand Manager Summary Report on Samsung S24 Feedback

**1. Sentiment Analysis**
The overall sentiment surrounding the Samsung S24 series is mixed but leans slightly negative. Users express emotions ranging from **disappointment** and **frustration** related to product performance, particularly regarding camera quality and software issues, to **excitement** and **satisfaction** with the S24's features and capabilities when comparing to their previous models. Comments often highlight discontent with the camera's image quality and user experience, contrasting it with their experiences with older models like the S21 or S20, which many feel performed better.

**2. Key Themes**
Key themes in discussions include:
- **Camera Quality**: A significant number of users express dissatisfaction with the camera performance, especially in low light conditions and color rendition.
- **Performance Comparisons**: Users frequently compare the S24 to previous models, often favoring older devices, which they believe outperformed the S24 in certain aspects.
- **Waiting for Future Models**: Many users are considering whether to purchase now or wait for the anticipated S25, leaning towards the latter due to perceived incremental upgrades and potential promotions.
- **Customization and Features**: Features like Good Lock for customization and battery health discussion are popular among users seeking to maximize their device's performance.

**3. Common Issues**
Recurring complaints include:
- **Poor Camera Performance**: Users report grainy and unsatisfactory photo quality, particularly in conditions they believe previous models handled better.
- **Software Glitches**: Reports of software issues, such as UI lag and unresponsive features.
- **Lack of Dramatic Upgrades**: Many express dissatisfaction with what they see as incremental updates from one model to the next, suggesting a stagnation in innovation.

**4. Suggestions and Recommendations**
To address consumer frustrations and enhance brand loyalty, consider the following actions:
- **Camera Software Updates**: Prioritize camera optimization updates addressing the current concerns about image quality.
- **Enhanced Pre-Release Testing**: Improve software testing protocols to reduce glitches and enhance device performance at launch.
- **Transparent Communication**: Engage with the community regarding expected improvements and features in software updates via forums or official channels.
- **Promotions and Trade-In Offers**: As discussions about waiting for the S25 show, a focus on attractive trade-in values and promotional offers could encourage sales and retention.

**5. Competitor Mentions**
Competitor products, particularly from Apple and Google (Pixel series), are frequently mentioned in discussions where users express how they consider switching due to design, camera capabilities, and ease of use. Users are particularly intrigued by Pixel's camera reputation and Apple's perceived stability and user experience.

**6. Engagement Insights**
- The post regarding the **fake S24** garnered significant attention with multiple users chiming in on the authenticity issue, highlighting the critical consumer awareness around counterfeit products.
- Posts celebrating new purchases of the S24 series received support and encouragement, suggesting a positive engagement when users are happy with their decisions.
- Threads discussing comparisons and experiences with older tech reveal a deeper user connection to their devices, emphasizing the emotional aspect of brand loyalty.

### Conclusion
The sentiment around the Samsung S24 series reveals significant opportunities for product improvement, particularly in camera technology and software stability. Addressing current user concerns through updates and engaging in active dialogue could help mitigate negative sentiment and foster greater loyalty. It's crucial to stay ahead of competitors by clearly communicating advancements and ensuring high-quality experiences for consumers transitioning from older models.

### Tests

In [34]:
from src import redditScrape
collected = redditScrape.collect_threads('Samsunggalaxy', 'S24')

from src import tokenValidate
all_content, messages = tokenValidate.ensure_tokens_within_limit(collected, prompt_template)

from src import getReport
report = getReport.generate_report(all_content, messages)

from IPython.display import Markdown
display(Markdown(report))